# 链
## 简单顺序链
简单的链可以将LLM的输出用作另一个的输入，适合分解任务

In [4]:
import json
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from dashscope import Generation

# 读取配置文件
with open("config.json", "r") as f:
    config = json.load(f)

# 将您的函数转换为Runnable对象
def chat_with_qwen(prompt):
    if hasattr(prompt, "to_string"):
        prompt_str = prompt.to_string()
    else:
        prompt_str = str(prompt)
        
    response = Generation.call(
        model=config['qwen_model'],
        prompt=prompt_str,
        api_key=config['api_key']
    )
    if response.status_code == 200:
        return response.output.text
    else:
        raise Exception(f"API调用失败: {response.code} - {response.message}")

# 将函数包装为Runnable对象
qwen_llm = RunnableLambda(chat_with_qwen)

# 创建提示模板
prompt1 = """
    想一个经典菜品
    %地区
    {location}

    你的回答：
"""

prompt_template1 = PromptTemplate(input_variables=["location"], template=prompt1)

# 创建第一个chain（使用管道方式）
location_chain = prompt_template1 | qwen_llm | StrOutputParser()

# 创建第二个提示模板
prompt2 = """
    给你一个菜品回复一个做菜流程
    %菜品
    {meal}

    你的回答：
"""

prompt_template2 = PromptTemplate(input_variables=["meal"], template=prompt2)

# 创建第二个chain
meal_chain = prompt_template2 | qwen_llm | StrOutputParser()

# 将两个chain连接起来
def process_location(location):
    # 获取第一个链的输出
    meal = location_chain.invoke({"location": location})
    print(f"生成的菜品: {meal}")
    # 将输出传给第二个链
    return meal_chain.invoke({"meal": meal})

final_chain = RunnableLambda(process_location)

# 调用链
output = final_chain.invoke("北京")
print(f"最终输出: {output}")

生成的菜品: 北京烤鸭
最终输出: 好的！以下是制作北京烤鸭的详细步骤：

---

### **北京烤鸭制作流程**

#### **1. 准备材料**
- 主料：肥嫩的北京填鸭一只（约2.5公斤）。
- 辅料：蜂蜜、白醋、麦芽糖（或白糖）、开水、盐、料酒。

#### **2. 清洗与处理鸭子**
1. 将鸭子宰杀后放血，拔毛并清洗干净。
2. 在鸭皮和肉之间分离脂肪层，但不要切断皮肤。
3. 去除内脏，用清水冲洗鸭腔，确保内部干净。
4. 用盐、料酒均匀涂抹鸭身内外，腌制15分钟去腥。

#### **3. 打气**
1. 将鸭子平放在案板上，从颈部切开一个小口。
2. 使用打气筒或手动方法，向鸭皮与肉之间打入空气，使鸭皮鼓起，形成饱满的形态。

#### **4. 挂钩与烫皮**
1. 用绳子将鸭子吊起，挂在通风处晾干表皮水分。
2. 锅中烧开水，用热水反复浇淋鸭皮表面，直到鸭皮紧绷且颜色变浅。

#### **5. 上色（脆皮水处理）**
1. 调制脆皮水：将蜂蜜、白醋和少量麦芽糖混合，加入少许开水搅拌均匀。
2. 用刷子将脆皮水上均匀地涂满鸭皮表面，然后再次挂起晾干，重复此过程至少3次。

#### **6. 烘烤**
1. 预热烤箱至200°C。
2. 将鸭子悬挂在烤炉内，用果木（如枣木或梨木）生火熏烤，同时不断调整鸭子的位置以保证受热均匀。
   - 烤制时间约为40-60分钟，期间需多次检查鸭皮颜色，避免烤焦。
3. 烤至鸭皮呈现金黄色且酥脆即可取出。

#### **7. 切片与装盘**
1. 将烤好的鸭子稍作冷却后，用锋利的刀具将鸭肉切成薄片，每片带皮带肉。
2. 将鸭片摆放在盘中，搭配葱丝、黄瓜条、甜面酱和荷叶饼一起食用。

#### **8. 上桌享用**
1. 将荷叶饼摊开，抹上一层甜面酱。
2. 放入适量葱丝、黄瓜条和鸭片，卷好后即可食用。

---

**小贴士：**
- 烤鸭的关键在于控制火候和时间，确保鸭皮酥脆、鸭肉鲜嫩。
- 可根据个人口味调整甜面酱的用量。

希望你能成功做出美味的北京烤鸭！


## 总结链
轻松浏览大量长文档并获取总结
不同的总结方法
- stuff 所有文档一次性进行总结
- map-reduce 每一段分别进行总结，然后给出最后的总结
- refine 每一段的总结都是上一段的总结加上本段的内容进行总结

In [ ]:
import os
import time
from typing import List

from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

from dashscope import Generation

# 创建与模型交互的函数
def chat_with_qwen(prompt):
    if hasattr(prompt, "to_string"):
        prompt_str = prompt.to_string()
    else:
        prompt_str = str(prompt)
        
    response = Generation.call(
        model='qwen-turbo',
        prompt=prompt_str,
        api_key=os.getenv('API_KEY')
    )
    if response.status_code == 200:
        return response.output.text
    else:
        raise Exception(f"API调用失败: {response.code} - {response.message}")

# 将函数包装为Runnable对象
qwen_llm = RunnableLambda(chat_with_qwen)

# 加载或定义你的文本
def load_text(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        return f.read()

# 将文本分割成小块
def split_text(text, chunk_size=1000, chunk_overlap=100):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
    )
    return text_splitter.create_documents([text])

# 创建映射提示模板 - 用于总结文档的每个部分
map_template = """
你是一位专业的文本总结专家。请对以下文本片段进行简洁有效的总结，保留关键信息和核心观点：

{text}

简明总结：
"""

map_prompt = PromptTemplate.from_template(map_template)

# 创建用于处理单个文档块的映射链
map_chain = (
    {"text": lambda x: x.page_content} 
    | map_prompt 
    | qwen_llm 
    | StrOutputParser()
)

# 创建合并提示模板 - 用于将各个部分的总结组合成一个完整总结
reduce_template = """

# 调用链
output = final_chain.invoke("北京")
print(f"最终输出: {output}")
最终总结：
"""

reduce_prompt = PromptTemplate.from_template(reduce_template)

# 创建合并链
def format_summaries(summaries):
    return "\n\n".join([f"总结 {i+1}:\n{s}" for i, s in enumerate(summaries)])

reduce_chain = (
    {"summaries": format_summaries} 
    | reduce_prompt 
    | qwen_llm 
    | StrOutputParser()
)

# 完整的MapReduce总结过程
def summarize_text_mapreduce(text_or_path, is_file=False, chunk_size=1000, max_chunks=None):
    # 加载文本
    if is_file:
        text = load_text(text_or_path)
    else:
        text = text_or_path
    
    print(f"文本长度: {len(text)} 字符")
    
    # 分割文本
    docs = split_text(text, chunk_size=chunk_size)
    if max_chunks and len(docs) > max_chunks:
        print(f"限制处理 {max_chunks}/{len(docs)} 个文档块")
        docs = docs[:max_chunks]
    else:
        print(f"处理全部 {len(docs)} 个文档块")
    
    # 映射阶段 - 处理每个文档块
    print("开始映射阶段...")
    start_time = time.time()
    summaries = []
    
    for i, doc in enumerate(docs):
        print(f"处理文档块 {i+1}/{len(docs)}...")
        summary = map_chain.invoke(doc)
        summaries.append(summary)
        # 打印每个块的总结
        print(f"块 {i+1} 总结: {summary[:50]}...")
    
    map_time = time.time() - start_time
    print(f"映射阶段完成，耗时: {map_time:.2f}秒")
    
    # 合并阶段 - 组合所有总结
    print("开始合并阶段...")
    start_time = time.time()
    
    # 如果总结太多，可能需要递归合并
    if len(summaries) > 10:
        # 分组处理
        print("总结数量过多，进行分组合并...")
        group_size = 5
        grouped_summaries = [summaries[i:i+group_size] for i in range(0, len(summaries), group_size)]
        intermediate_summaries = []
        
        for i, group in enumerate(grouped_summaries):
            print(f"合并组 {i+1}/{len(grouped_summaries)}...")
            group_summary = reduce_chain.invoke(group)
            intermediate_summaries.append(group_summary)
        
        final_summary = reduce_chain.invoke(intermediate_summaries)
    else:
        final_summary = reduce_chain.invoke(summaries)
    
    reduce_time = time.time() - start_time
    print(f"合并阶段完成，耗时: {reduce_time:.2f}秒")
    
    total_time = map_time + reduce_time
    print(f"总结完成，总耗时: {total_time:.2f}秒")
    
    return final_summary

# 直接使用函数
def summarize_document(text_or_path, is_file=False):
    return summarize_text_mapreduce(text_or_path, is_file)

# 使用示例
if __name__ == "__main__":
    # 可以是文件路径
    summary = summarize_document("xhs.txt", is_file=True)
    
    # # 或者直接输入文本
    # long_text = """
    # [这里是你想要总结的长文本...]
    # MapReduce是Google提出的一个软件架构，用于大规模数据集的并行运算。概念"Map（映射）"和"Reduce（归约）"，
    # 以及它们的主要思想，都是从函数式编程语言借来的，同时包含了从矢量编程语言借来的特性。
    
    # Map(映射)是指对一组数据中的每个数据项分别执行同样的操作。例如，假设有一组整数{1,2,3,4,5}，给每个整数加1的结果就是{2,3,4,5,6}。
    
    # Reduce(归约)指的是对一个序列的元素进行合并的操作，比如对一个整数列表{2,3,4,5,6}求和，就会得到20。
    
    # 在文本处理中，Map阶段可以对每个文本块单独总结，而Reduce阶段则将这些总结合并成一个完整的总结。
    
    # 这种方法适用于处理超长文本，因为我们可以先分解问题（映射阶段），然后再组合结果（归约阶段），
    # 有效避开了模型对长文本处理能力的限制。
    
    # 文本处理的MapReduce过程具体步骤如下：
    # 1. 将长文本分割成较小的块
    # 2. 对每个文本块单独进行总结（Map）
    # 3. 将所有总结合并成一个最终总结（Reduce）
    
    # 这种方法也适用于其他大型语言处理任务，如文档问答、信息提取等。
    # """
    
    # summary = summarize_document(long_text)
    print("\n最终总结:")
    print(summary)

文本长度: 106765 字符
处理全部 120 个文档块
开始映射阶段...
处理文档块 1/120...


Exception: API调用失败: InvalidApiKey - Invalid API-key provided.